Hmm okay, so we're up to the main proessing step. Essentially everything to this point has just been pre-processing data to get it into a nice format. Here we actually take that data and use the plate model to rotate it through time in order to populate the ocean basins. We also have a subduction zone test, because we need to know when our 'points' are subducted and turned off. I took this from the workflow of Williams (2019) (see https://github.com/siwill22/agegrid-0.1, and https://zenodo.org/record/3271360 and Williams et al. (in review). In principal the functionality is also quite similar to work by Krister Karlson at Oslo (Karlson et al. 2019).

I think eventually it will be better if I can move most of this to a function, but for now it's here in it's dirty mess (sorry).

References

Karlsen, K.S, Domeier, M., Gaina, C., & Conrad, C. (2019, September 1). TracerTectonics (Version 1.0). Zenodo. http://doi.org/10.5281/zenodo.3383154

Merdith, A.S., Atkins, S.E., and Tetley, M.G. (2019). Tectonic controls on carbon and serpentinite storage in subducted upper oceanic lithosphere for the past 320 Ma. Frontiers: Earth Science.

Williams, S.E., (2019, July 8). siwill22/agegrid-0.1 v1-alpha (Version v1-alpha). Zenodo. http://doi.org/10.5281/zenodo.3271360

In [2]:
import numpy as np
import pandas as pd
import pygplates
import pickle
import time as tme
import itertools
import filter_and_reconstruct_points

/Users/Andrew/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1066: UserWarning: Duplicate key in file "/Users/Andrew/.matplotlib/matplotlibrc", line #3
  (fname, cnt))
/Users/Andrew/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1066: UserWarning: Duplicate key in file "/Users/Andrew/.matplotlib/matplotlibrc", line #4
  (fname, cnt))


In [7]:
datadir = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/Merdith_2019_Frontiers/Sample_Data/'
savedir = '/Users/Andrew/Documents/PhD/Scripts/Scripts_Output/Serpentinisation/output/'

rotfile = '%sMatthews_etal_GPC_2016_410-0Ma_GK07.rot' % datadir
topologies = ['%sMatthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies.gpmlz' % datadir,
              '%sMatthews_etal_GPC_2016_Paleozoic_PlateTopologies.gpmlz' % datadir]

rotation_model = pygplates.RotationModel(rotfile)


In [4]:
# import previously made file of crustal properties
#separate load so faster if we need to recheck stuff
#C_storage = pickle.load(open('%s/C_storage_2020-01-23_PMSR.p' % datadir, 'rb'))
C_storage = pickle.load(open('%s/C_storage_2020-03-13_PMSR.p' % datadir, 'rb'))


In [11]:
#apologise for this mess
#POSR or PMSR?
a
POSR = False
#PMSR = True
#set some parameters up

#resolution of grids
resolution = 1 #in degrees
#final timestep
end_time = 0
#timestep
timestep = 1
#make rasters at these times
times = np.arange(400,end_time-timestep,-(timestep))

#inititate loop to extract data
start_time = tme.time()
for time in times:
    #define all our times (previous time, current time, next time)
    print time
    next_timestep = time - timestep
    previous_timestep = time + timestep
    
    #resolve topologies for rotating points at this timestep, and at the previous timestep
    curr_resolved_topologies = []
    prev_resolved_topologies = []
    pygplates.resolve_topologies(topologies, rotation_model, curr_resolved_topologies, time)
    pygplates.resolve_topologies(topologies, rotation_model,prev_resolved_topologies, previous_timestep)
    
    #if not first timestep, then we need to rotate points from the previous timestep to the current timestep
    if time != np.max(times):
        #time = 100, next_timestep = 99
        #next becomes current, current becomes previous
        prev_lat, prev_lon = curr_lat, curr_lon
        curr_lat, curr_lon, curr_zvals = next_lat, next_lon, next_zvals
        #partition plates and points
        prev_plate_partitioner, prev_plateIDs = curr_plate_partitioner, curr_plateIDs
        curr_plate_partitioner = pygplates.PlatePartitioner(curr_resolved_topologies, rotation_model)
        #get new set of plate IDs for current timestep
        curr_plateIDs = filter_and_reconstruct_points.get_plate_ID_of_points(curr_plate_partitioner,
                                                                             curr_lat,
                                                                             curr_lon)

        #returns none if it falls into a topology gap then filters to remove it
        curr_lat, curr_lon, curr_zvals = filter_and_reconstruct_points.filter_points_Nones(curr_lat, 
                                                                                           curr_lon,
                                                                                           curr_plateIDs,
                                                                                           curr_zvals)
        #lets get the resolved boundares of our points to test against
        #there should be no \'Nones\' present, since we got rid of them just before
        
        resolved_topologies_containing_prev_points = filter_and_reconstruct_points.find_polygon_of_points(prev_lat,
                                                                            prev_lon,
                                                                            prev_resolved_topologies)
        #filter current points
        curr_lat = filter(None, curr_lat)
        curr_lon = filter(None, curr_lon)
        curr_zvals = filter(None, curr_zvals)
        curr_plateIDs = filter(None, curr_plateIDs)

        for ind, i in enumerate(prev_plateIDs):
            #there are more plate IDs in current that previous, because we have all the new points attached
            #but because they are appended, they occur at the end, so indices of prev_plateIDs should correspond
            #to indices of curr_plateIDs until we run out of prev_plateIDs
            prev_inputlatLon = pygplates.LatLonPoint(prev_lat[ind], prev_lon[ind])
            prev_latLonPoint = pygplates.convert_lat_lon_point_to_point_on_sphere(prev_inputlatLon)
            curr_inputlatLon = pygplates.LatLonPoint(curr_lat[ind], curr_lon[ind])
            curr_latLonPoint = pygplates.convert_lat_lon_point_to_point_on_sphere(curr_inputlatLon)
            prev_point_rotation = rotation_model.get_rotation(time, i, previous_timestep)
            curr_point_rotation = rotation_model.get_rotation(time, curr_plateIDs[ind], previous_timestep)

            #this is our subduction test, after Williams et al. (in review)
            if curr_plateIDs[ind] != i:
                collision = filter_and_reconstruct_points.detect_collision(timestep,
                                             curr_latLonPoint,
                                             prev_latLonPoint,
                                             curr_point_rotation,
                                             prev_point_rotation,
                                             curr_plateIDs[ind],
                                             resolved_topologies_containing_prev_points[ind],
                                             i)
                if collision == True:
                    #if they've collided then these points can be None'd
                    curr_lat[ind] = None
                    curr_lon[ind] = None
                    curr_zvals[ind] = None
                    curr_plateIDs[ind] = None
                    
        #originally some points were slipping through, so double sure #NoNones
        curr_lat = filter(None, curr_lat)
        curr_lon = filter(None, curr_lon)
        curr_zvals = filter(None, curr_zvals)
        curr_plateIDs = filter(None, curr_plateIDs)
        
        print '# of points', len(curr_lat)
        
        #now we can export our data
        if time <= 200:
            filter_and_reconstruct_points.exporting(curr_lon,curr_lat,curr_zvals, time, savedir, POSR)
           #exporting values to xyz table
        #now to rotate through time to next timestep
        #re-initiate the next time step lat/lon/zvals, as we want to overwrite all previous 'nexts'
        #(which are 'current')
        #with the actual next timestep
        next_lat = []
        next_lon = []
        next_zvals = []
        for ind, i in enumerate(curr_plateIDs):

                #print ind
                #we have already accounted for points that have been subducted, so lets rotate them through time to generate next
                #NB we output points for next timestep
                curr_inputlatLon = pygplates.LatLonPoint(curr_lat[ind], curr_lon[ind])
                curr_latLonPoint = pygplates.convert_lat_lon_point_to_point_on_sphere(curr_inputlatLon)
                next_point_rotation = rotation_model.get_rotation(next_timestep, i, time)
                next_reconstructed_point = next_point_rotation * curr_latLonPoint
                tmp_lat = next_reconstructed_point.to_lat_lon_list()[0][0]
                tmp_lon = next_reconstructed_point.to_lat_lon_list()[0][1]
                #print tmp_lat, tmp_lon, next_reconstructed_point
                next_lat.append(tmp_lat)
                next_lon.append(tmp_lon)
                next_zvals.append(curr_zvals[ind])
    
    #get new data at timestep (OR) initiate first data if at max(times)
    MOR_lat = []
    MOR_lon = []
    MOR_zvals = []
    MOR_plateIDs = []
    init_curr_lat = []
    init_curr_lon = []
    init_curr_plateIDs = []
    for i,j,k,l,m,n,o,p,q,r,s,t,u in itertools.izip(C_storage[time]['Start-Lat-Point'],
                                                   C_storage[time]['End-Lat-Point'],
                                                   C_storage[time]['Start-Lon-Point'],
                                                   C_storage[time]['End-Lon-Point'],
                                                   C_storage[time]['RightPlate'],
                                                   C_storage[time]['LeftPlate'],
                                                   C_storage[time]['C-prod'],
                                                   C_storage[time]['C-prod_vertical'],
                                                   C_storage[time]['Serpentinites_raw'],
                                                   C_storage[time]['Serpentinites_raw_vertical'],
                                                   C_storage[time]['Thickness Mean'],
                                                   C_storage[time]['Vertical Area Mean'],
                                                   C_storage[time]['Spreading Rate']):


        a = s[0] #total thick
        b = s[1] #upper thick
        c = s[2] #lower thick
        d = s[3] #trans thick
        e = s[4] #dyke thick
        f = s[5] #gabbro thick
        
        #this is if we're using the POSR.
        if POSR:
            if time > 160:
                print'here'
                #print 'using POSR'
                #use vertical values if old, gets undone in subduction flux, but not perfectly
                #this lets us detect them later on more easily
                #divide by two so we don't double dip
    
                o = p * 1000000/2    #C
                q = r * 1000000/2    #serp
                a = t[0] * 1000000/2 #total thick
                b = t[1] * 1000000/2 #upper thick
                c = t[2] * 1000000/2 #lower thick
                d = t[3] * 1000000/2 #trans thick
                e = t[4] * 1000000/2 #dyke thick
                f = t[5] * 1000000/2 #gabbro thick
            
        points = []
        points.append((i,k))
        points.append((j,l))
        polyline = pygplates.PolylineOnSphere(points)
        
        #get stage rotations of plate IDs i.e. m and n
        line_rotation_right = rotation_model.get_rotation(next_timestep, int(m), time)
        line_rotation_left = rotation_model.get_rotation(next_timestep, int(n), time)
        reconstructed_polyline_right = line_rotation_right * polyline
        reconstructed_polyline_left = line_rotation_left * polyline
        tessellated_polyline_right = reconstructed_polyline_right.to_tessellated(np.radians(2))
        tessellated_polyline_left = reconstructed_polyline_left.to_tessellated(np.radians(2))
        latlon_list_right = tessellated_polyline_right.to_lat_lon_list()
        latlon_list_left = tessellated_polyline_left.to_lat_lon_list()
        recon_lats_right, recon_lons_right = [i[0] for i in latlon_list_right], [i[1] for i in latlon_list_right]
        recon_lats_left, recon_lons_left = [i[0] for i in latlon_list_left], [i[1] for i in latlon_list_left]
        
        #if time is max we have a separate class for 'initial values', this is just to initiate the sequence and
        #is unused after the first loop, as we keep appending new values onto 'curr_lat etc.'
        if time == np.max(times):
            #print h
            init_curr_tessellated_polyline_right = polyline.to_tessellated(np.radians(2))
            init_curr_tessellated_polyline_left = polyline.to_tessellated(np.radians(2))
            init_curr_latlon_list_right = init_curr_tessellated_polyline_right.to_lat_lon_list()
            init_curr_latlon_list_left = init_curr_tessellated_polyline_left.to_lat_lon_list()
            init_curr_recon_lats_right, init_curr_recon_lons_right = [i[0] for i in init_curr_latlon_list_right], [i[1] for i in init_curr_latlon_list_right]
            init_curr_recon_lats_left, init_curr_recon_lons_left = [i[0] for i in init_curr_latlon_list_left], [i[1] for i in init_curr_latlon_list_left]

            for i in range(len(init_curr_recon_lats_right)):
                init_curr_lat.append(init_curr_recon_lats_right[i])
                init_curr_lat.append(init_curr_recon_lats_left[i])
                init_curr_lon.append(init_curr_recon_lons_right[i])
                init_curr_lon.append(init_curr_recon_lons_left[i])
                #init_curr_plateIDs.append(m)
                #init_curr_plateIDs.append(n)
                #update arrays with data at new timestep
                #right then left
                
        for i in range(len(recon_lats_right)):
            MOR_lat.append(recon_lats_right[i])
            MOR_lat.append(recon_lats_left[i])
            MOR_lon.append(recon_lons_right[i])
            MOR_lon.append(recon_lons_left[i])
            #have to do zvals twice, once for right ridge, second time for left ridge
            #important, as otherwise only half of your ocean is populated
            MOR_zvals.append([o,q,a,b,c,d,e,f,u,time])
            MOR_zvals.append([o,q,a,b,c,d,e,f,u,time])

            #plateIDs as well :-\
            MOR_plateIDs.append(int(m))
            MOR_plateIDs.append(int(n))
    if time == np.max(times):
        #to initialise at max time, we get the partitioner and plateIDs of the first step of rotated points
        #this isn\'t technically correct, but as we are only dealing with 1 Ma worth of points, they should fall
        #within the correct ocean basin unless at initialisation time the ridge is being subducted (not in our case)
        curr_plate_partitioner = pygplates.PlatePartitioner(curr_resolved_topologies, rotation_model)
        curr_plateIDs = MOR_plateIDs
        curr_lat, curr_lon = init_curr_lat, init_curr_lon
        next_lat = (MOR_lat)
        next_lon = (MOR_lon)
        next_zvals = (MOR_zvals)

    if time != np.max(times):
        print 'whats been added', len(MOR_lat)
        next_lat.extend(MOR_lat)
        next_lon.extend(MOR_lon)
        next_zvals.extend(MOR_zvals)
        
            
        print len(next_lat)
        #print len(next_lat)

    print("--- %s seconds to timestep ---" % (tme.time() - start_time))
print("--- %s seconds overall ---" % (tme.time() - start_time))

#fin


165
--- 4.50797796249 seconds to timestep ---
164
# of points 774
whats been added 768
1542
--- 9.08390712738 seconds to timestep ---
--- 9.08423495293 seconds overall ---


In [17]:
next_zvals[0][1]*next_zvals[0][2]*1000000/2

2378442.151106552